In [7]:
import os, sys
import argparse
import time

import xml.etree.ElementTree as ET

from sumolib import checkBinary

import traci
import traci.constants as tc

import matplotlib.pyplot as plt

In [8]:
import gym

In [9]:
if sys.platform == "win32":
    # windows, win32
    from sumolib import checkBinary
else:
    # mac, darwin
    if 'SUMO_HOME' in os.environ:
        tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
        sys.path.append(tools)
    else:
        sys.exit("please declare environment variable 'SUMO_HOME'")
    
import traci

In [10]:
import curbside
import utils

In [11]:
net_xml = "seattle.net.xml"
add_xml = "seattle.add.xml"
rou_xml = "seattle.trips.xml"

In [ ]:
class SeattleEnv(gym.Env):
    def __init__(self):
        self.net_xml = "seattle.net.xml"
        self.add_xml = "seattle.add.xml"
        self.rou_xml = "seattle.trips.xml"
        self.curb_ids = _init_curb_ids():
        self.curbs = _init_curbs()
    
    def _init_curb_ids(self):
        curb_ids = []
        root = ET.parse(self.add_xml).getroot()
        for child in root.iter('additional'):
            for kid in child.iter('parkingArea'):
                curb_ids.append(kid.get('id'))
        return curb_ids
                
    def _init_curbs(self):
        curbs = {}
        for curb_id in self.curb_ids:
            curbs.append(curbside.SmartCurbside(1, 'seattle.add.xml', 'seattle.net.xml', curb_id, ['passenger', 'taxi'], road_network))
        return curbs

In [ ]:
class A2CAgent:
    def __init__(self):
        self.config = None

In [ ]:
class Policy:
    def __init__(self, curb_ids):
        self.agents = {}
        for curb_id in curb_ids:
            self.agents[curb_id] = A2CAgent()

In [4]:
def simulate(seconds = 3600, gui = False):
    """
    Main function that controls the simulation
    """

    road_network = utils.create_graph('seattle.net.xml')
    
    # create agent curbs
    curb_ids = []
    root = ET.parse(add_xml).getroot()
    for child in root.iter('additional'):
        for kid in child.iter('parkingArea'):
            curb_ids.append(kid.get('id'))
    
    curbs = []
    for curb_id in curb_ids:
        curbs.append(curbside.SmartCurbside(1, 'seattle.add.xml', 'seattle.net.xml', curb_id, ['passenger', 'taxi'], road_network))
        
    for curb in curbs:
        curb.find_neighborhood(road_network)
    
    reroute_num = 0

    if gui:
        sumoBinary = checkBinary('sumo-gui')
    else:
        sumoBinary = checkBinary('sumo')

    traci.start([sumoBinary, "-c", "seattle.sumocfg"])
    

    control_start_time = 60 # arranging existing vehicle until this time (seconds) to start to control curbs or collecting training samples
    
    control_time_window = 60 # take action every 60 seconds
    
    step = 0 - control_start_time
    MAX_step = control_start_time + control_start_time
    
    # simulation
    start = time.time()
    while traci.simulation.getMinExpectedNumber() > 0 and step < MAX_step:
        traci.simulationStep()
        step += 1
        
        for curb in curbs:
            # update neighbors info
            curb._update_nearby_curb()
            
            # check if there is any vehicle enters
            v_enter = set(traci.edge.getLastStepVehicleIDs(curb.edge)) - curb.moving_vehicle
            
            # no need to subtract v_leave, because v_leave ideally should be maintained per curb, but now inner loop is on curbs
            # v_enter -= v_leave
            
            for veh in v_enter:
                # new vehicle is for sure not confirmed, no need to check confirmed=True
                # if vehicle has next stops and currect edge is one of them - this vehicle should be considered for current edge
                # also check vehicle is not stopped - during parking right before departure getNextStops() will return original travel plan, which shouldn't happen
                if traci.vehicle.getNextStops(veh) and curb.id in [item[2] for item in traci.vehicle.getNextStops(veh)] and not traci.vehicle.isStopped(veh):
                    # if the trips is not ending and the current edge is the parking stop
                    # v_leave should be excluded
                    if any(traci.vehicle.getVehicleClass(veh) in s for s in curb.vclass) and len(curb.occupied_vehicle) < curb.capacity:
                        # if can park, add to occupied set : planned + parked
                        curb.occupied_vehicle.add(veh)
                    else:
                        # cannot park at this edge, reroute
                        # item._reroute_choice() returns (curb_id, distance) tuple
                        traci.vehicle.rerouteParkingArea(veh, curb._reroute_choice()[0])
                        reroute_num += 1
                        
            # v_leave : parked vehicle of last time step - parked vehicle at this time step
            v_leave = curb.parked_vehicle - set(traci.parkingarea.getVehicleIDs(curb.id))
            # remove v_leave from occupied set : planned + parked
            curb.occupied_vehicle -= v_leave
            # update parked vehicle set
            curb.parked_vehicle = set(traci.parkingarea.getVehicleIDs(curb.id))
            # update moving vehicles on the hosting edge
            curb.moving_vehicle = set(traci.edge.getLastStepVehicleIDs(curb.edge))
            
            if step % control_time_window == 0: 
                if step == 0:
                    c_p, o_p, c_d, o_d = curb.cap_passenger, curb.occ_passenger, curb.cap_delivery, curb.occ_passenger
                    state = [c_p, o_p, c_d, o_d]
                    action = curb.policy(state)
                    
                else:
            
            action = 
            
        
        # calculate running time of an interation
        iteration_time = time.time() - start
            
    # enter False to disconnect before SUMO finishes
    traci.close()

    # flush out intermmediate results
    sys.stdout.flush()
    
    return reroute_num, running_time